# Split Movie Frames

The first step in training a cryo-CARE network is to split the direct detector acquisition movies. To that end we will first align the frames with [MotionCor2](https://www.nature.com/articles/nmeth.4193) and then split the frames into even and odd acquistions.

__Note__: This only works if you have downloaded MotionCor2 and placed it in the `example` directory. For more details check out at the [README](https://github.com/juglab/cryoCARE_simg/blob/master/README.md#get-motioncor2).

In [1]:
from glob import glob
import mrcfile
import os
from os.path import join, basename

import numpy as np
from tqdm import tqdm

from matplotlib import pyplot as plt

from utils import *

In [2]:
# In this example we use the downloaded tomogram 'Tomo110', 
# kindly provided by Mareike Jordan from the Pigino Lab at 
# MPI-CBG, Dresden. 
data_path = '/data/Tomo110/frames/'

In [3]:
# Read the list of mrc-files and display them.
files = glob(data_path + '*.mrc')
files

['/data/Tomo110/frames/54_Tomo110_-42.0_Apr04_16.01.22.mrc',
 '/data/Tomo110/frames/39_Tomo110_-12.0_Apr04_15.23.20.mrc',
 '/data/Tomo110/frames/46_Tomo110_-26.0_Apr04_15.55.28.mrc',
 '/data/Tomo110/frames/32_Tomo110_2.0_Apr04_15.28.23.mrc',
 '/data/Tomo110/frames/25_Tomo110_16.0_Apr04_15.33.26.mrc',
 '/data/Tomo110/frames/57_Tomo110_-48.0_Apr04_16.03.35.mrc',
 '/data/Tomo110/frames/27_Tomo110_12.0_Apr04_15.32.00.mrc',
 '/data/Tomo110/frames/20_Tomo110_26.0_Apr04_15.37.06.mrc',
 '/data/Tomo110/frames/04_Tomo110_58.0_Apr04_15.48.58.mrc',
 '/data/Tomo110/frames/47_Tomo110_-28.0_Apr04_15.56.10.mrc',
 '/data/Tomo110/frames/22_Tomo110_22.0_Apr04_15.35.40.mrc',
 '/data/Tomo110/frames/44_Tomo110_-22.0_Apr04_15.53.53.mrc',
 '/data/Tomo110/frames/35_Tomo110_-4.0_Apr04_15.26.10.mrc',
 '/data/Tomo110/frames/55_Tomo110_-44.0_Apr04_16.02.07.mrc',
 '/data/Tomo110/frames/30_Tomo110_6.0_Apr04_15.29.48.mrc',
 '/data/Tomo110/frames/07_Tomo110_52.0_Apr04_15.46.37.mrc',
 '/data/Tomo110/frames/36_Tomo110_-

In [4]:
# We can also execute imod-commands by putting a '!' infront of 
# the command.
!header {files[0]}


 RO image file on unit   1 : /data/Tomo110/frames/54_Tomo110_-42.0_Apr04_16.01.22.mrc     Size=     556211 K

 Number of columns, rows, sections .....    7420    7676      10
 Map mode ..............................    0   (byte)                     
 Start cols, rows, sects, grid x,y,z ...    0     0     0    7420   7676     10
 Pixel spacing (Angstroms)..............   2.355      2.355      2.355    
 Cell angles ...........................   90.000   90.000   90.000
 Fast, medium, slow axes ...............    X    Y    Z
 Origin on x,y,z .......................    0.000       0.000       0.000    
 Minimum density .......................   0.0000    
 Maximum density .......................   140.00    
 Mean density ..........................   4.9266    
 tilt angles (original,current) ........   0.0   0.0   0.0   0.0   0.0   0.0
 Space group,# extra bytes,idtype,lens .        0        0        0        0

     1 Titles :
SerialEMCCD: Dose frac. image, scaled by 16.00  r/f 1   04

In [5]:
# Here we read out the pixel spacing from the imod-command.
# Pixel spacing (Angstroms) according to the header
p = !header -p {files[0]}
pixel_spacing = float(p[1].split()[0])
print('Pixel Spacing in Angstroms:', pixel_spacing)

Pixel Spacing in Angstroms: 2.3555


## CUDA-9.2

To run MotionCor2 we have to change the `PATH` to include `CUDA-9.2` instead of `CUDA-9.0`. 

The command `nvcc --version` shows which `CUDA` is currently active.

In [6]:
# Setting the environment variable PATH
%env PATH=/usr/local/bin:/usr/local/IMOD/bin:/usr/local/nvidia/bin:/usr/local/cuda-9.2/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
# Execute the nvcc command
!nvcc --version

env: PATH=/usr/local/bin:/usr/local/IMOD/bin:/usr/local/nvidia/bin:/usr/local/cuda-9.2/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Tue_Jun_12_23:07:04_CDT_2018
Cuda compilation tools, release 9.2, V9.2.148


## Align & Split

Each file gets aligned with MotionCor2 and the aligned stack is written to a `tmp` directory. Then the aligned stack is loaded and splitted in the two halves (even/odd) and written to the corresponding directories.

In [7]:
# Create directories
if not os.path.isdir('tmp'):
    os.mkdir('tmp')
if not os.path.isdir(join(data_path, 'even')):
    os.mkdir(join(data_path, 'even'))
if not os.path.isdir(join(data_path, 'odd')):
    os.mkdir(join(data_path, 'odd'))

In [8]:
def align_and_split(files):
    for f in tqdm(files):
        # Align the frames with MotionCor2 and write out the aligned stack with '-OutStack 1'
        !/notebooks/MotionCor2_1.2.1-Cuda92 -InMrc {f} -OutMrc tmp/aligned.mrc -Patch 5 5 5 -OutStack 1 >> motioncor2.log
        # Since we process image by image only one aligned stack is available here
        aligned_stack = mrcfile.open(glob('tmp/*_Stk.mrc')[0], permissive=True)
        # Save even frames
        save_mrc(join(data_path, 'even', basename(f)), np.sum(aligned_stack.data[::2], axis=0), pixel_spacing)
        # Save odd frames
        save_mrc(join(data_path, 'odd', basename(f)), np.sum(aligned_stack.data[1::2], axis=0), pixel_spacing)
        # Remove aligned files
        remove_files('tmp', extension='.mrc')

In [9]:
# MotionCor2 will create a log file 
align_and_split(files)

  0%|          | 0/65 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/mrcfile/mrcinterpreter.py:208: RuntimeWarning: Map ID string not found - not an MRC file, or file is corrupt
  warnings.warn(msg, RuntimeWarning)
100%|██████████| 65/65 [20:31<00:00, 19.27s/it]


## CUDA-9.0

After running MotionCor2 we change the `PATH` back to `CUDA-9.0` which will be used for the network training. 

In [10]:
# Change the environment variable PATH back
%env PATH=/usr/local/bin:/usr/local/IMOD/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
!nvcc --version

env: PATH=/usr/local/bin:/usr/local/IMOD/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Sep__1_21:08:03_CDT_2017
Cuda compilation tools, release 9.0, V9.0.176


## Remove `tmp` Directory

In [11]:
os.removedirs('tmp')